In [1]:
import os

import dotenv
import pandas as pd
import requests


dotenv.load_dotenv()

# Define the headers, including any necessary authentication
headers = {
    "Content-Type": "application/json",
    "ApiKey": os.getenv("ApiKey"),
}


In [2]:
def gql_query(query, variables=None):
    headers = {"Content-Type": "application/json"}
    if os.getenv("STASHDB_API_KEY"):
        headers["Apikey"] = os.getenv("STASHDB_API_KEY")
    response = requests.post(
        "https://stashdb.org/graphql",
        json={"query": query, "variables": variables},
        headers=headers,
    )
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(
            f"Query failed with status code {response.status_code}: {response.text}"
        )


In [8]:
# Define the GraphQL query
query = """
    query QueryScenes($stash_ids: [ID!]!, $page: Int!) {
        queryScenes(
            input: {
                performers: {
                    value: $stash_ids,
                    modifier: INCLUDES
                },
                per_page: 25,
                page: $page
            }
        ) {
            scenes {
                id
                title
                details
                release_date
                urls {
                    url
                    site {
                        name
                        url
                    }
                }
                studio {
                    id
                    name
                    parent {
                        id
                        name
                    }
                }
                images {
                    id
                    url
                }
                performers {
                    performer {
                        id
                        name
                    }
                }
                duration
                code
                tags {
                    id
                    name
                }
            }
            count
        }
    }
"""

performer_stash_id = "e7167957-a37f-476e-83f8-86427676b4ff"
page = 1
data = gql_query(
    query, {"stash_ids": performer_stash_id, "page": page}
)

# Parse the JSON response into a DataFrame
df = pd.json_normalize(data["data"]["queryScenes"]["scenes"])  # Use pd.json_normalize to flatten the data if it's nested

In [ ]:
from pathlib import Path


# Read the GraphQL query from a file
with Path("stashdb-graphql/findSceneByFingerprint.graphql").open("r") as file:
    query = file.read()

# Execute the query as before
response = requests.post("https://stashdb.org/graphql", json={"query": query, "variables": variables}, headers=headers)

# Parse the JSON response into a DataFrame
data = response.json()
single_scene_df = pd.json_normalize(data["data"]["findSceneByFingerprint"])  # Use pd.json_normalize to flatten the data if it's nested


In [ ]:
# Read the GraphQL query from a file
with Path("stashdb-graphql/studio_scenes.graphql").open("r") as file:
    query = file.read()

variables = {
    "input": {
        "page": 1,
        "per_page": 20,
        "sort": "DATE",
        "direction": "DESC",
        "parentStudio": "324ea274-1afb-4f80-aa66-c6ddb52b9b56"
    }
}

# Execute the query as before
response = requests.post("https://stashdb.org/graphql", json={"query": query, "variables": variables}, headers=headers)

# Parse the JSON response into a DataFrame
data = response.json()
studio_scenes_df = pd.json_normalize(data["data"]["queryScenes"]["scenes"])  # Use pd.json_normalize to flatten the data if it's nested


In [ ]:
# Read the GraphQL query from a file
with Path("stashdb-graphql/studio_scenes.graphql").open("r") as file:
    query = file.read()

# Initialize variables
page = 1
all_scenes = []
total_scenes = None

while total_scenes is None or len(all_scenes) < total_scenes:
    variables = {
        "input": {
            "page": page,
            "per_page": 20,
            "sort": "DATE",
            "direction": "DESC",
            "parentStudio": "324ea274-1afb-4f80-aa66-c6ddb52b9b56"
        }
    }

    # Execute the query
    response = requests.post("https://stashdb.org/graphql", json={"query": query, "variables": variables}, headers=headers)
    data = response.json()

    # Append the scenes to all_scenes
    all_scenes.extend(data["data"]["queryScenes"]["scenes"])

    # Get the total number of scenes if it's not already set
    if total_scenes is None:
        total_scenes = data["data"]["queryScenes"]["count"]

    # Increment the page number
    page += 1

# Convert all_scenes to a DataFrame
studio_scenes_df = pd.json_normalize(all_scenes)  # Use pd.json_normalize to flatten the data if it's nested